In [1]:
import pandas as pd
from utils import dataManager as dm
from utils import callbackFunctions as cf


# # To autoreload modules
# %load_ext autoreload
# %autoreload 2

structuresDf = cf.loadStructuresDf('data/structures.json')


In [2]:
dataPath = ".\data\originalData\data_SD1.xlsx"
wfa = dm.readMetricsDataForGenes(dataPath)

dataPath = ".\data\originalData\data_SD2.xlsx"
pv = dm.readMetricsDataForGenes(dataPath)

In [3]:
# LOAD GENE EXPRESSION DATA
dataPath = ".\data\gene_expression_ABA_energy.csv"
ish_en =  pd.read_csv(dataPath, index_col=0)
ish_en.columns = pd.to_numeric(ish_en.columns)

selMetric = wfa['energy']
ishData = ish_en

dataPath = ".\data\originalData\data_SD4.xlsx"
geneCorrData = pd.read_excel(dataPath, header=0, index_col=0)

In [14]:
selGene = 11382
g = geneCorrData.loc[geneCorrData['gene_AGEA_id']==selGene]
g.iloc[0]['gene_name']

'aggrecan'

In [4]:
g = geneCorrData.loc[geneCorrData['gene_AGEA_id']==selGene]

g = g.T.reset_index()
g.columns = ['Parameter', 'Value']
g

NameError: name 'selGene' is not defined

In [77]:
selGene = 11382


metricDf = pd.DataFrame(wfa['energy'])
metricDf.columns=['metric']
metric = metricDf.reset_index()

ishData.index.name = 'mid'
ish = pd.DataFrame(ishData.loc[selGene])
ish.columns = ['geneExp']

merged = metric.join(ish, on='mid')

structureDf = cf.loadStructuresDf('data/structures.json')
structureDf = structureDf[['acronym', 'name', 'rgb_plotly']]
merged = merged.join(structureDf, on='mid')

merged = merged.set_index(['coarse','mid'])

merged = merged.drop(1009, level='mid')

merged

metric   geneExp  acronym  \
coarse mid                                      
315    184        1.178936  0.337825      FRP   
       985        1.185993  0.115369      MOp   
       993        0.960004  0.083421      MOs   
       353        2.884739  0.215506    SSp-n   
       329        2.445807  0.279295  SSp-bfd   
...                    ...       ...      ...   
512    1049       0.062824  0.102786       FL   
       989        3.975008  1.280930       FN   
       91         4.722340  1.789552       IP   
       846        4.308272  1.421883       DN   
       589508455  2.633070       NaN     VeCB   

                                                      name        rgb_plotly  
coarse mid                                                                    
315    184                   Frontal pole, cerebral cortex    rgb(38,143,69)  
       985                              Primary motor area    rgb(31,157,90)  
       993                            Secondary motor area    rgb(31,157,90)  
       353                Primary somatosensory area, nose   rgb(24,128,100)  
       329        Primary somatosensory area, barrel field   rgb(24,128,100)  
...                                                    ...               ...  
512    1049                                      Flocculus  rgb(255,252,145)  
       989                               Fastigial nucleus  rgb(255,253,188)  
       91                               Interposed nucleus  rgb(255,253,188)  
       846                                 Dentate nucleus  rgb(255,253,188)  
       589508455               Vestibulocerebellar nucleus  rgb(255,253,188)  

[314 rows x 5 columns]

In [87]:
import numpy as np
import plotly.express as px
from plotly import graph_objects as go

# fig = go.Figure()

fig = cf.make_GeneScatter()
# remove all present data
fig.data = []

# Draw a Scatter trace for each area in the dataFrame
for coarse, new_df in merged.groupby(level='coarse'):
    thisTrace = go.Scatter(
        x = new_df['geneExp'], y = new_df['metric'],
        name = structureDf.loc[coarse]['name'],
        marker_color = new_df['rgb_plotly'],
        mode='markers',
        marker = dict(
            size=13,
            line_width=1,
            opacity=0.85,
        ),
        customdata  = np.stack((new_df['name'], new_df['acronym']), axis=-1),
        hovertemplate= "<b>%{customdata[1]}</b>" + "<br>" +
            "<i>%{customdata[0]}</i>" + "<br>" + 
            f"{structureDf.loc[coarse]['name']}" + "<br>" +
            f"<b>Staining Metric</b>:" "%{x:.3f}" + "<br>" + 
            f"<b>Gene Expression</b>:" + "%{y:.3f}" +
            "<extra></extra>",
        
    )
    fig.add_trace(thisTrace)

# Sort traces alphabetically
fig.data = sorted(fig.data, key=lambda d: d['name']) 
# fig.update_xaxes(type="log")
# fig.update_yaxes(type="log")

fig.show()


In [8]:
# LOAD GENE EXPRESSION DATA
dataPath = ".\data\gene_expression_ABA_energy.csv"
ish_en =  pd.read_csv(dataPath, index_col=0)
ish_en.columns = pd.to_numeric(ish_en.columns)
ish_en

,184,985,993,353,329,337,345,369,361,182305689,...,1017,1025,1033,1041,1049,989,91,846,589508455,1009
gene_id,,,,,,,,,,,,,,,,,,,,,
11126.0,0.473943,0.500063,0.380301,1.354610,0.752251,0.371575,0.689385,0.610790,0.531021,0.611769,...,0.848387,0.584759,1.031300,1.785600,0.827087,0.295098,0.437803,0.331696,NaN,NaN
11134.0,0.368271,0.118375,0.082076,0.099143,0.090373,0.169150,0.099486,0.129184,0.093695,0.126556,...,0.203991,0.151322,0.400241,0.141211,0.105338,0.299928,0.240518,0.213599,NaN,NaN
11138.0,5.821550,11.560200,9.973270,7.209290,8.615720,11.209600,8.495910,7.132290,8.312070,8.866190,...,5.951530,4.349760,7.871250,4.053100,2.933160,8.635530,11.964900,10.323800,NaN,NaN
11139.0,1.437350,0.365165,0.321158,0.154990,0.210551,0.631820,0.152069,0.180905,0.428857,0.170930,...,0.403761,0.447509,0.666701,0.484117,0.722483,0.249710,0.582740,0.358063,NaN,NaN
11140.0,1.264680,0.758946,0.735843,0.632790,0.544552,0.857853,0.636478,0.657498,0.927717,0.634649,...,0.473478,0.520546,1.830750,1.566280,1.278710,0.258622,0.769895,0.886037,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028068.0,0.030953,0.027163,0.060801,0.026112,0.034533,0.003623,0.026801,0.040057,0.076633,0.023490,...,0.079932,0.205561,0.084048,0.048442,0.050757,0.011558,0.010867,0.000847,NaN,NaN
1031710.0,0.562739,2.465340,2.384840,7.132790,6.344190,1.878710,4.561200,1.872130,1.629110,3.155870,...,14.445000,9.030140,7.215770,11.533400,12.597700,2.933350,4.497100,1.639100,NaN,NaN
1033266.0,1.567025,1.343336,1.101622,1.213564,0.886782,0.869605,1.236550,0.959510,0.640915,1.313487,...,0.688793,0.678058,1.451988,0.973112,0.917408,3.221695,3.216665,2.920635,NaN,NaN


In [9]:
dataPath = ".\data\originalData\data_SD4.xlsx"

df = pd.read_excel(dataPath, header=0, index_col=0)

dictList = []
for idx, row in df.iterrows():
    temp = {'label':idx,'value':row['gene_AGEA_id']}
    dictList.append(temp)

dictList


[{'label': 'Adssl1', 'value': 11352},
 {'label': 'Lifr', 'value': 16651},
 {'label': 'Pnkd', 'value': 35974},
 {'label': 'Fam43a', 'value': 86045},
 {'label': 'Acan', 'value': 11382},
 {'label': 'Kank4', 'value': 88970},
 {'label': 'Dtnbp1', 'value': 60877},
 {'label': 'Atmin', 'value': 87898},
 {'label': 'Gm12429', 'value': 130280},
 {'label': 'B830028H17Rik*', 'value': 150110},
 {'label': 'Ddt', 'value': 12982},
 {'label': 'Ccdc126', 'value': 37166},
 {'label': 'Kcnab3', 'value': 16272},
 {'label': 'Cpeb3', 'value': 84034},
 {'label': 'Kcns3', 'value': 88419},
 {'label': 'Asb13', 'value': 80270},
 {'label': 'Pfkfb3', 'value': 82070},
 {'label': 'Cacybp', 'value': 12086},
 {'label': 'Myo19', 'value': 42039},
 {'label': 'Limk1', 'value': 16656},
 {'label': 'Paqr4', 'value': 52339},
 {'label': 'Lgi2', 'value': 89814},
 {'label': 'Ndufb5', 'value': 41889},
 {'label': 'Msantd2', 'value': 87971},
 {'label': 'Vwa5b2', 'value': 108506},
 {'label': 'Scrt1', 'value': 82032},
 {'label': 'Lamp5'

In [10]:
df

,gene_AGEA_id,gene_entrez_id,gene_name,spearman_rho,p_value,FDR,adjusted p_value (bonferroni)
gene_acronym,,,,,,,
Adssl1,11352,11565.0,adenylosuccinate synthetase like 1,0.632670,2.224075e-32,3.923046e-28,3.923046e-28
Lifr,16651,16880.0,leukemia inhibitory factor receptor,0.609203,1.573082e-29,1.387380e-25,2.774760e-25
Pnkd,35974,56695.0,paroxysmal nonkinesiogenic dyskinesia,0.607231,2.663811e-29,1.566232e-25,4.698697e-25
Fam43a,86045,224093.0,"family with sequence similarity 43, member A",0.573612,1.235043e-25,5.446230e-22,2.178492e-21
Acan,11382,11595.0,aggrecan,0.571899,1.850719e-25,6.528966e-22,3.264483e-21
...,...,...,...,...,...,...,...
Pdlim1,33425,54132.0,PDZ and LIM domain 1 (elfin),NaN,1.000000e+00,1.000000e+00,1.763900e+04
Tmem50b,53815,77975.0,transmembrane protein 50B,NaN,1.000000e+00,1.000000e+00,1.763900e+04
Pla1a,59218,85031.0,phospholipase A1 member A,NaN,1.000000e+00,1.000000e+00,1.763900e+04


In [27]:
pd.concat([wfa['energy'], ish_en.loc[11140]], axis=1)

,energy,11140.0
184,1.178936,1.264680
985,1.185993,0.758946
993,0.960004,0.735843
353,2.884739,0.632790
329,2.445807,0.544552
...,...,...
91,4.722340,0.769895
846,4.308272,0.886037
589508455,2.633070,NaN
1009,0.231939,NaN


In [3]:
energy = dfDict['mid'].xs('energy', axis=1,level='params')

mean = energy.mean(axis=1)
mean

coarse  coarse_acro   mid        mid_acro    
315     Isocortex     184        FRP             1.178936
                      985        MOp             1.185993
                      993        MOs             0.960004
                      353        SSp-n           2.884739
                      329        SSp-bfd         2.445807
                                                   ...   
512     CB            989        FN              3.975008
                      91         IP              4.722340
                      846        DN              4.308272
                      589508455  VeCB            2.633070
1009    fiber tracts  1009       fiber tracts    0.231939
Length: 315, dtype: float64

In [39]:
import matplotlib as mpl
from matplotlib import cm
norm = mpl.colors.Normalize(vmin=0, vmax=3, clip=False)

mapper = cm.ScalarMappable(norm=norm, cmap='Reds')
rgb = mapper.to_rgba(mean)


res = pd.DataFrame(data=mean, columns=['Mean'])
res['rgb'] = [(x[0],x[1],x[2]) for x in rgb]
rgb = [f'rgb({x[0]},{x[1]},{x[2]})' for x in rgb]
res['color'] = rgb
res.index = res.index.droplevel(['coarse', 'coarse_acro','mid'])


res.loc['PO'].color

'rgb(1.0,0.9607843137254902,0.9411764705882353)'